In [4]:
import spacy
import os

from spacy import displacy
from textblob import TextBlob
import aspect_based_sentiment_analysis as absa

nlp = spacy.load('en_core_web_sm')

### Helpers

In [5]:
# nlp.add_pipe(nlp.create_pipe('merge_noun_chunks'))

def merge_nouns(doc):
    """If two consecutive tokens are nouns, it concatenates them into one \
    representing one aspect term.
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns:
        new_text {str}: text for retokenization after nouns have been merged
    """
    
    i=0
    new_text = ''
    while i<len(doc):
        if doc[i].dep_ == 'compound':
            compound_noun = doc[i].text + doc[i+1].text
            new_text = new_text + ' ' + compound_noun
            i += 2
        else:
            new_text = new_text + ' ' + doc[i].text
            i += 1
    return new_text

In [37]:
def tokenize(text):
    """Parses the given text using nlp pipeline
    
    Arguments:
        text {str}: Text to be parsed
    
    Returns:
        doc {obj}: nlp document object
    """
    
    doc_raw = nlp(text)
    return doc_raw

In [18]:
def plot_dependencies(doc):
    """Plots the dependencies in the nlp document
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns: 
        displacy plot
    """
    
    displacy.serve(doc, style="dep")

In [9]:
def get_pos_tags(doc):
    """Prints the pos tags for each token in the document
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns:
        token.pos_ {unicode}: POS tag of each token
    """
    
    for token in doc:
        print(token.text, token.pos_)

In [10]:
def get_aspect_terms(doc):
    """This function returns the root noun present in noun chunks of the document.
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns:
        aspects {list{str}}: list of root nouns as aspects
    """
                                            
    aspects = [(chunk.root.text) for chunk in doc.noun_chunks if chunk.root.pos_ == 'NOUN']
    return aspects

In [11]:
def get_sentiment_terms(doc):
    """This function return the adjectives and verbs that are not stopwords \
    or punctuations, indicating a descriptive/polarized word.
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns:
        sentiment_terms {list{str}}: list of sentiment terms
    """
    
    sentiment_terms = []
    if doc.is_parsed:
        sentiment_terms.append([token.lemma_ for token in doc if \
                                (not token.is_stop and not token.is_punct \
                                 and (token.pos_ == "ADJ" or token.pos_ == "VERB"))])
    else:
        sentiment_terms.append('') 
    return sentiment_terms

In [12]:
def get_dependencies(doc):
    """Prints the dependency tree of the document
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns:
        token.text {unicode}: Verbatim text content
        token.dep_ {unicode}: Syntactic dependency relation
        token.head.text {unicode}: The syntactic parent, or “governor”, of a token
        token.head.pos_ {unicode}: POS tag of the governor
        children {list}: list of children of a token
        
    """
    
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])

In [13]:
def get_opinion_pairs(doc):
    """This function returns the opinion pairs based on pre-defined rules.
    
    Arguments:
        doc {obj}: nlp document object
    
    Returns:
        opinion_pairs {list{tuple}}: list of tuples consisiting of (aspect, opinion)
    """
    
    opinion_pairs = []
    for token in doc:
        if token.dep_ == 'nsubj' and TextBlob(token.head.text).polarity > 0.4:
            opinion_pairs.append((token.text, token.head.text))
        elif token.dep_ == 'dobj' and (token.head.pos_ == 'ADJ' or TextBlob(token.head.text).polarity > 0.4):
            opinion_pairs.append((token.text, token.head.text))
        elif token.dep_ == 'amod' and token.head.pos_ == 'ADJ':
            opinion_pairs.append((token.text, token.head.text))
    return opinion_pairs

In [68]:
def get_attention_gradient(text):
    nlp = absa.load(pattern_recognizer=absa.probing.AttentionGradientProduct())
    service, conversation = nlp(text, aspects=['service', 'conversation'])
    display(absa.probing.explain(service.examples[0]))
    display(absa.probing.explain(conversation.examples[0]))

### Implementation

In [38]:
text = "Slow service, but the waiter were friendly. He kept us engaged in conversation"

In [75]:
def func_1(text):
    doc1 = tokenize(text)
    new_text = merge_nouns(doc1)
    doc = tokenize(new_text)
    return doc

res = func_1(text)

TypeError: __call__() missing 1 required positional argument: 'aspects'

In [39]:
doc1 = tokenize(text)
new_text = merge_nouns(doc1)
doc = tokenize(new_text)

In [41]:
get_aspect_terms(doc)

['service', 'waiter', 'conversation']

In [42]:
get_sentiment_terms(doc)

[['slow', 'friendly', 'keep', 'engage']]

In [43]:
get_opinion_pairs(doc)

[('service', 'Slow')]

In [69]:
get_attention_gradient(text)

Some weights of the model checkpoint at absa/classifier-rest-0.1 were not used when initializing BertABSClassifier: ['extractor', 'dropout_4711']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertABSClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertABSClassifier were not initialized from the model checkpoint at absa/classifier-rest-0.1 and are newly initialized: ['dropout_341']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
plot_dependencies(doc)

/Users/a0s01w2/Downloads/anaconda3/envs/absa_spacy/lib/python3.8/runpy.py:194: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  return _run_code(code, main_globals, None,



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
